<a href="https://colab.research.google.com/github/bascoul/TP_Deep_Learning/blob/master/Part2_Debiasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="center">
  <td align="center"><a target="_blank" href="http://introtodeeplearning.com">
        <img src="http://introtodeeplearning.com/images/colab/mit.png" style="padding-bottom:5px;" />
      Visit MIT Deep Learning</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/aamini/introtodeeplearning/blob/master/lab2/Part2_Debiasing.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/aamini/introtodeeplearning/blob/master/lab2/Part2_Debiasing.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Copyright Information

In [0]:
# Copyright 2020 MIT 6.S191 Introduction to Deep Learning. All Rights Reserved.
# 
# Licensed under the MIT License. You may not use this file except in compliance
# with the License. Use and/or modification of this code outside of 6.S191 must
# reference:
#
# © MIT 6.S191: Introduction to Deep Learning
# http://introtodeeplearning.com
#

# Laboratoire 2: Computer Vision

# Partie 2 : Débiaiser les systèmes de détection faciale

Dans la deuxième partie du laboratoire, nous explorerons deux aspects importants de l'apprentissage approfondi appliqué : la détection faciale et le biais algorithmique. 

Le déploiement de systèmes d'IA équitables et impartiaux est essentiel à leur acceptation à long terme. Considérons la tâche de la détection faciale : étant donné une image, s'agit-il d'une image d'un visage ?  Cette tâche apparemment simple, mais extrêmement importante, est sujette à des biais algorithmiques importants parmi certaines données démographiques. 

Dans ce laboratoire, nous allons étudier [une approche récemment publiée] (http://introtodeeplearning.com/AAAI_MitigatingAlgorithmicBias.pdf) pour traiter les biais algorithmiques. Nous allons construire un modèle de détection faciale qui apprend les *variables latentes* sous-jacentes aux ensembles de données d'images de visages et les utiliser pour ré-échantillonner de manière adaptative les données d'entraînement, atténuant ainsi tout biais qui pourrait être présent afin d'entraîner un modèle *biaisé*.


Lancez le bloc de code suivant pour une courte vidéo de Google qui explore comment et pourquoi il est important de prendre en compte les biais lorsqu'on pense à l'apprentissage machine :

In [0]:
import IPython
IPython.display.YouTubeVideo('59bMh59JQDo')

Commençons par installer les dépendances pertinentes :

In [0]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf

import IPython
import functools
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Download and import the MIT 6.S191 package
!pip install mitdeeplearning
import mitdeeplearning as mdl

## 2.1 Ensembles de données

Nous utiliserons trois ensembles de données dans ce laboratoire. Afin d'entraîner nos modèles de détection faciale, nous aurons besoin d'un ensemble de données d'exemples positifs (c'est-à-dire de visages) et d'un ensemble de données d'exemples négatifs (c'est-à-dire de choses qui ne sont pas des visages). Nous utiliserons ces données pour entraîner nos modèles à classer les images comme étant des visages ou non. Enfin, nous aurons besoin d'un ensemble de données de test pour les images de visages. Comme nous sommes préoccupés par le *biais* potentiel de nos modèles appris par rapport à certaines données démographiques, il est important que l'ensemble de données de test que nous utilisons ait une représentation égale des données démographiques ou des caractéristiques d'intérêt. Dans ce laboratoire, nous allons prendre en compte la couleur de la peau et le sexe. 

1.   **Données d'entraînement positives** : [CelebA Dataset] (http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html). Un ensemble à grande échelle (plus de 200K images) de visages de célébrités.   
2.   **Données négatives sur la formation** : [ImageNet](http://www.image-net.org/). De nombreuses images dans de nombreuses catégories différentes. Nous prendrons des exemples négatifs de diverses catégories non humaines. 
Système de classification des types de peau [Fitzpatrick Scale](https://en.wikipedia.org/wiki/Fitzpatrick_scale), chaque image étant étiquetée comme "plus claire" ou "plus foncée".

Commençons par importer ces ensembles de données. Nous avons écrit une classe qui fait un peu de prétraitement des données pour importer les données d'entraînement dans un format utilisable.

In [0]:
# Obtenez les données de formation : les deux images de CelebA et ImageNet
path_to_training_data = tf.keras.utils.get_file('train_face.h5', 'https://www.dropbox.com/s/l5iqduhe0gwxumq/train_face.h5?dl=1')
# Installer un TrainingDatasetLoader en utilisant l'ensemble de données téléchargées
loader = mdl.lab2.TrainingDatasetLoader(path_to_training_data)

Nous pouvons examiner la taille de l'ensemble des données relatives à l'apprentissage et prendre un lot de taille 100 :

In [0]:
number_of_training_examples = loader.get_train_size()
(images, labels) = loader.get_batch(100)

Jouez avec l'affichage d'images pour avoir une idée de ce à quoi ressemblent réellement les données d'apprentissage !

In [0]:
### Examen de l'ensemble des données d'apprentissage de CelebA ###

#@title Changez les curseurs pour voir des exemples d'apprentissage positifs et négatifs ! { run: "auto" }

face_images = images[np.where(labels==1)[0]]
not_face_images = images[np.where(labels==0)[0]]

idx_face = 23 #@param {type:"slider", min:0, max:50, step:1}
idx_not_face = 9 #@param {type:"slider", min:0, max:50, step:1}

plt.figure(figsize=(5,5))
plt.subplot(1, 2, 1)
plt.imshow(face_images[idx_face])
plt.title("Face"); plt.grid(False)

plt.subplot(1, 2, 2)
plt.imshow(not_face_images[idx_not_face])
plt.title("Not Face"); plt.grid(False)

### Penser aux biais

N'oubliez pas que nous formerons nos classificateurs de détection faciale sur le vaste ensemble de données bien documenté de CelebA (et ImageNet), puis nous évaluerons leur précision en les testant sur un ensemble de données de test indépendant. Notre objectif est de construire un modèle qui s'entraîne sur CelebA *et* qui atteint une grande précision de classification sur l'ensemble des données de test pour toutes les catégories démographiques, et de montrer ainsi que ce modèle ne souffre d'aucun biais caché. 

Que voulons-nous dire exactement quand nous disons qu'un classificateur est biaisé ? Pour formaliser cela, nous devons penser aux [*variables latentes*] (https://en.wikipedia.org/wiki/Latent_variable), variables qui définissent un ensemble de données mais ne sont pas strictement observées. Comme défini dans la conférence sur la modélisation générative, nous utiliserons le terme *espace latent* pour désigner les distributions de probabilité des variables latentes susmentionnées. En rassemblant ces idées, nous considérons qu'un classificateur est *biaisé* si sa décision de classification change après qu'il ait vu quelques caractéristiques latentes supplémentaires. Cette notion de biais peut être utile à garder à l'esprit dans le reste du laboratoire. 

## 2.2 CNN pour la détection faciale 

Tout d'abord, nous allons définir et former une CNN sur la tâche de classification des visages, et évaluer sa précision. Ensuite, nous évaluerons la performance de nos modèles débiatisés par rapport à cette CNN de référence. Le modèle CNN a une architecture relativement standard qui consiste en une série de couches convolutionnelles avec normalisation par lots, suivie de deux couches entièrement connectées pour aplatir la sortie de convolution et générer une prédiction de classe. 

### Définir et entraîner le modèle CNN

Comme nous l'avons fait dans la première partie du laboratoire, nous allons définir notre modèle CNN, puis nous entraîner sur les ensembles de données CelebA et ImageNet en utilisant la classe `tf.GradientTape` et la méthode `tf.GradientTape.gradient`.

In [0]:
### Définir le modèle CNN ###

n_filters = 12 # nombre de base pour les filtres convolutionels

'''Fonction pour définir un modèle CNN standard'''
def make_standard_classifier(n_outputs=1):
  Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
  BatchNormalization = tf.keras.layers.BatchNormalization
  Flatten = tf.keras.layers.Flatten
  Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

  model = tf.keras.Sequential([
    Conv2D(filters=1*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),
    
    Conv2D(filters=2*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),

    Conv2D(filters=4*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Conv2D(filters=6*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Flatten(),
    Dense(512),
    Dense(n_outputs, activation=None),
  ])
  return model

standard_classifier = make_standard_classifier()

Maintenant, formons la CNN standard !

In [0]:
### Former le CNN standard ###

# Hyperparamètres d'apprentissage
batch_size = 32
num_epochs = 2  # à garder petit pour calculer plus vite
learning_rate = 5e-4

optimizer = tf.keras.optimizers.Adam(learning_rate) # définir notre optimiseur
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) # pour enregistrer l'évolution de la perte
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # nettoyer si elles existent

@tf.function
def standard_train_step(x, y):
  with tf.GradientTape() as tape:
    # introduire les images dans le modèle
    logits = standard_classifier(x) 
    # Calculer la perte
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

  # Backpropagation
  grads = tape.gradient(loss, standard_classifier.trainable_variables)
  optimizer.apply_gradients(zip(grads, standard_classifier.trainable_variables))
  return loss

# La boucle d'apprentissage!
for epoch in range(num_epochs):
  for idx in tqdm(range(loader.get_train_size()//batch_size)):
    # Saisissez un lot de données d'apprentissage et propagez-les dans le réseau
    x, y = loader.get_batch(batch_size)
    loss = standard_train_step(x, y)

    # Enregistrer la perte et tracer l'évolution de la perte en fonction de l'apprentissage
    loss_history.append(loss.numpy().mean())
    plotter.plot(loss_history.get())

### Évaluer la performance du CNN standard

Ensuite, évaluons les performances de classification de notre CNN standard formé par CelebA sur l'ensemble des données d'apprentissage.


In [0]:
### Evaluation du CNN standard ###

# DONNEES D'APPRENTISSAGE
# Evaluer sur un sous-ensemble de CelebA+Imagenet
(batch_x, batch_y) = loader.get_batch(5000)
y_pred_standard = tf.round(tf.nn.sigmoid(standard_classifier.predict(batch_x)))
acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))

print("Standard CNN accuracy on (potentially biased) training set: {:.4f}".format(acc_standard.numpy()))

Nous évaluerons également nos réseaux sur un ensemble de données de test indépendant contenant des visages qui n'ont pas été vus pendant la formation. Pour les données de test, nous examinerons la précision de la classification sur quatre catégories démographiques différentes, en nous basant sur l'échelle de la peau de Fitzpatrick et sur des étiquettes basées sur le sexe : homme à la peau foncée, femme à la peau foncée, homme à la peau claire et femme à la peau claire. 

Examinons quelques exemples de visages dans l'ensemble de tests.

In [0]:
### Charger l'ensemble de données de dataset et afficher les examples ###

test_faces = mdl.lab2.get_test_faces()
keys = ["Light Female", "Light Male", "Dark Female", "Dark Male"]
for group, key in zip(test_faces,keys): 
  plt.figure(figsize=(5,5))
  plt.imshow(np.hstack(group))
  plt.title(key, fontsize=15)

Maintenant, évaluons la probabilité que chacune de ces données démographiques soit classée comme un visage en utilisant le classificateur standard CNN que nous venons de former. 

In [0]:
### Evaluer le CNN standard sur l'ensemble de test ### 

standard_classifier_logits = [standard_classifier(np.array(x, dtype=np.float32)) for x in test_faces]
standard_classifier_probs = tf.squeeze(tf.sigmoid(standard_classifier_logits))

# Tracer les précisions de prévision par groupe démographique
xx = range(len(keys))
yy = standard_classifier_probs.numpy().mean(1)
plt.bar(xx, yy)
plt.xticks(xx, keys)
plt.ylim(max(0,yy.min()-yy.ptp()/2.), yy.max()+yy.ptp()/2.)
plt.title("Standard classifier predictions");

Jetez un coup d'œil aux précisions de ce premier modèle dans ces quatre groupes. Qu'observez-vous ? Considérez-vous ce modèle comme biaisé ou non biaisé ? Quelles sont les raisons pour lesquelles un modèle formé peut avoir des précisions biaisées ? 

## 2.3 Atténuer le biais algorithmique

Les déséquilibres dans les données d'entraînement peuvent entraîner un biais algorithmique indésirable. Par exemple, la majorité des visages dans CelebA (notre jeu d'entraînement) sont ceux de femmes à la peau claire. Par conséquent, un classificateur formé sur CelebA sera mieux à même de reconnaître et de classer les visages ayant des caractéristiques similaires à celles-ci, et sera donc biaisé.

Comment pourrions-nous surmonter cela ? Une solution naïve - et sur ce point, de nombreuses entreprises et organisations l'adoptent - consisterait à annoter différentes sous-classes (c'est-à-dire les femmes à la peau claire, les hommes avec des chapeaux, etc.) dans les données d'apprentissage, puis à égaliser manuellement les données concernant ces groupes.

Mais cette approche présente deux inconvénients majeurs. Premièrement, elle nécessite l'annotation de quantités massives de données, ce qui n'est pas évolutif. Deuxièmement, elle exige que nous sachions quels préjugés potentiels (par exemple, la race, le sexe, la pose, l'occlusion, les chapeaux, les lunettes, etc. ) à rechercher dans les données. Par conséquent, l'annotation manuelle peut ne pas saisir toutes les différentes caractéristiques qui sont déséquilibrées dans les données d'entraînement.

Au lieu de cela, nous allons en fait **apprendre** ces caractéristiques d'une manière impartiale et non supervisée, sans avoir besoin d'annotation, et ensuite former un classificateur de manière équitable en ce qui concerne ces caractéristiques. Dans le reste de ce laboratoire, c'est exactement ce que nous ferons.

## 2.4 Auto-codeur variationnel (VAE) pour l'apprentissage de la structure latente

Comme vous l'avez vu, la précision du CNN varie selon les quatre données démographiques que nous avons examinées. Pour comprendre pourquoi, il suffit de considérer l'ensemble de données sur lequel le modèle a été formé, CelebA. Si certaines caractéristiques, telles que la peau foncée ou les chapeaux, sont *rares* dans CelebA, le modèle peut finir par être biaisé par rapport à celles-ci suite à l'apprentissage avec un ensemble de données biaisées. C'est-à-dire que la précision de sa classification sera pire sur les visages qui ont des caractéristiques sous-représentées, comme les visages à la peau foncée ou les visages avec des chapeaux, par rapport aux visages dont les caractéristiques sont bien représentées dans les données d'entraînement ! C'est un problème. 

Notre objectif est de former une version *débiaisée* de ce classificateur - une version qui tienne compte des disparités potentielles dans la représentation des caractéristiques dans les données d'entraînement. Plus précisément, pour construire un classificateur facial débiaisé, nous allons former un modèle qui **apprend une représentation de l'espace latent sous-jacent** aux données d'entraînement du visage. Le modèle utilise ensuite ces informations pour atténuer les biais indésirables en échantillonnant des visages aux caractéristiques rares, comme la peau foncée ou les chapeaux, *plus fréquemment* pendant l'apprentissage. La principale exigence de conception de notre modèle est qu'il puisse apprendre un *codage* des caractéristiques latentes dans les données du visage d'une manière entièrement *non supervisée*. Pour y parvenir, nous nous tournerons vers les auto-codeurs variationnels (VAE).

[Le concept de VAE] (http://kvfrans.com/content/images/2016/08/vae.jpg)

Comme le montrent le schéma ci-dessus et le cours 4, les VAE s'appuient sur une structure d'encodeur-décodeur pour apprendre une représentation latente des données d'entrée. Dans le contexte de la vision par ordinateur, le réseau d'encodeurs prend les images d'entrée, les encode en une série de variables définies par une moyenne et un écart-type, puis puise dans les distributions définies par ces paramètres pour générer un ensemble de variables latentes échantillonnées. Le réseau de décodeurs "décode" ensuite ces variables pour générer une reconstruction de l'image originale, qui est utilisée pendant la formation pour aider le modèle à identifier les variables latentes qu'il est important d'apprendre. 

Formalisons deux aspects clés du modèle VAE et définissons les fonctions pertinentes pour chacun.


### Comprendre les VAE : la fonction de perte

En pratique, comment former un VAE ? En apprenant l'espace latent, nous contraignons les moyennes et les écarts-types à suivre approximativement une unité gaussienne. Rappelons que ce sont des paramètres appris, et qu'ils doivent donc être pris en compte dans le calcul de la perte, et que la partie décodeur de l'EVA utilise ces paramètres pour produire une reconstruction qui doit correspondre étroitement à l'image d'entrée, qui doit également être prise en compte dans la perte. Cela signifie que nous aurons deux termes dans notre fonction de perte VAE :

1.  **La perte latente ($L_{KL}$)** : qui mesure le degré de correspondance des variables latentes apprises avec une unité gaussienne et est définie par la divergence de Kullback-Leibler (KL).
2.   **La Perte de reconstruction ($L_{x}{(x,\hat{x})}$)** : qui mesure la précision avec laquelle les sorties reconstruites correspondent à l'entrée et est donnée par la norme $L^1$ de l'image d'entrée et de sa sortie reconstruite.  

Les équations pour ces deux pertes sont fournies ci-dessous :

$$ L_{KL}(\mu, \sigma) = \frac{1}{2}\sum\limites_{j=0}^{k-1}\small{(\sigma_j + \mu_j^2 - 1 - \log{\sigma_j})} $$

$$ L_{x}{(x,\hat{x})} = ||x-\hat{x}||_1 $$ 

Ainsi pour la perte de VAE que nous avons : 

$$ L_{VAE} = c\cdot L_{KL} + L_{x}{(x,\hat{x})} $$

où $c$ est un coefficient de pondération utilisé pour la régularisation. 

Nous sommes maintenant prêts à définir notre fonction de perte de VAE :

In [0]:
### Definir la fonction de perte VAE ###

''' Fonction pour calculer la perte VAE connaissant :
      une entrée x, 
      une sortie reconstruite x_recon, 
      les moyennes encodées mu, 
      un logarithme encodé de l'écart-type logsigma, 
      un paramètre de poids pour la perte latente kl_weight
'''
def vae_loss_function(x, x_recon, mu, logsigma, kl_weight=0.0005):
  # A FAIRE : Définir la perte latente. Notez que cela est donné dans l'équation pour L_{KL}
  # dans le bloc de texte directement ci-dessus
  latent_loss = # A FAIRE

  # A FAIRE : Définissez la perte de reconstruction comme la différence absolue 
  # moyenne, pixel par pixel, entre l'entrée et la reconstruction.
  # Conseil : vous devrez utiliser tf.reduce_mean, et fournir un argument d'axe
  # qui spécifie les dimensions à réduire. Par exemple, la perte de reconstruction
  # doit être sur les dimensions de la hauteur, de la largeur et du canal de l'image.
  # https://www.tensorflow.org/api_docs/python/tf/math/reduce_mean
  reconstruction_loss = # A FAIRE

  # A FAIRE : Définir la perte de VAE. Notez que cela est donné dans l'équation 
  # pour L_{VAE} dans le bloc de texte directement au-dessus
  vae_loss = # A FAIRE
  
  return vae_loss

Super ! Maintenant que nous avons une idée plus concrète du fonctionnement des VAE, voyons comment nous pouvons tirer parti de cette structure en réseau pour former un classificateur facial *déficient*.

### Comprendre les VAE : le reparamétrage 

Comme vous vous en souvenez peut-être, les VAE utilisent un "truc de reparamétrage" pour l'échantillonnage des variables latentes apprises. Au lieu que le codeur VAE génère un seul vecteur de nombres réels pour chaque variable latente, il génère un vecteur de moyennes et un vecteur d'écarts types qui sont contraints de suivre approximativement des distributions gaussiennes. Nous échantillonnons ensuite à partir des écarts types et ajoutons la moyenne pour obtenir notre vecteur latent échantillonné. En formalisant ceci pour une variable latente $z$ où nous échantillonnons $\epsilon \sim \mathcal{N}(0,(I))$ nous avons : 

$$ z = \mathbb{\mu} + e^{\gauche(\frac{1}{2} \cdot \log{\Sigma}\droit)}\circ \epsilon $$

où $\mu$ est la moyenne et $\Sigma$ est la matrice de covariance. Cela est utile car cela nous permettra de définir précisément la fonction de perte pour la VAE, de générer des variables latentes échantillonnées au hasard, d'obtenir une meilleure généralisation du réseau, **et** de rendre notre réseau VAE complet différenciable afin qu'il puisse être formé par rétropropagation. Tout à fait puissant !

Définissons une fonction pour mettre en œuvre l'opération d'échantillonnage VAE :

In [0]:
### Reparamétrisation du VAE ###

"""Astuce de reparamétrage par échantillonnage à partir d'une unité isotrope gaussienne
# Arguments
    z_mean, z_logsigma (tenseur): moyenne et log de l'écart-type de la distribution latente (Q(z|X))
# Retour
    z (tenseur): vecteur latent échantillonné
"""
def sampling(z_mean, z_logsigma):
  # Par défaut, random.normal est "standard" (c'est-à-dire mean=0 et std=1.0)
  batch, latent_dim = z_mean.shape
  epsilon = tf.random.normal(shape=(batch, latent_dim))

  # A FAIRE : Définir le calcul de la reparamétrisation !
  # Notez que l'équation est donnée dans le bloc de texte immédiatement supérieur.
  z = # A FAIRE
  return z

## 2.5 Débiaisement de l'auto-codeur variationnel (DB-VAE)

Maintenant, nous allons utiliser l'idée générale de l'architecture VAE pour construire un modèle, appelé [*debiasing variational autoencoder*] (https://lmrt.mit.edu/sites/default/files/AIES-19_paper_220.pdf) ou DB-VAE, pour atténuer les biais (potentiellement) inconnus présents dans l'idée de formation. Nous formerons notre modèle DB-VAE à la tâche de détection faciale, exécuterons l'opération de déviation pendant la formation, évaluerons l'ensemble de données PPB et comparerons sa précision à notre modèle CNN original, biaisé.    

### Le modèle DB-VAE

L'idée clé derrière cette approche de débiabilisation est d'utiliser les variables latentes apprises via une VAE pour rééchantillonner de manière adaptative les données de CelebA pendant l'apprentissage. Plus précisément, nous modifierons la probabilité qu'une image donnée soit utilisée pendant la formation en fonction de la fréquence à laquelle ses caractéristiques latentes apparaissent dans l'ensemble de données. Ainsi, les visages présentant des caractéristiques plus rares (comme une peau sombre, des lunettes de soleil ou un chapeau) devraient être plus susceptibles d'être échantillonnés pendant l'entraînement, tandis que la probabilité d'échantillonnage des visages présentant des caractéristiques surreprésentées dans l'ensemble de données d'entraînement devrait diminuer (par rapport à un échantillonnage aléatoire uniforme dans les données d'entraînement). 

Un schéma général de l'approche DB-VAE est présenté ici :

![DB-VAE](https://raw.githubusercontent.com/aamini/introtodeeplearning/2019/lab2/img/DB-VAE.png)Notez que l'équation est donnée dans le bloc de texte immédiatement supérieur.

Rappelons que nous voulons appliquer notre DB-VAE à un problème de *classification supervisée* : la tâche de détection faciale. Il est important de noter que la partie encodeur de l'architecture DB-VAE produit également une seule variable supervisée, $z_o$, correspondant à la prédiction de classe -- visage ou non visage. Habituellement, les VAE ne sont pas formés pour produire des variables supervisées (comme une prédiction de classe) ! Il s'agit là d'une autre distinction essentielle entre la DB-VAE et une VAE traditionnelle. 

Gardez à l'esprit que nous voulons seulement apprendre la représentation latente des *faces*, car c'est ce contre quoi nous nous battons en fin de compte, même si nous formons un modèle sur un problème de classification binaire. Nous devons nous assurer que, **pour les visages**, notre modèle DB-VAE apprend à la fois une représentation des variables latentes non supervisées, capturées par la distribution $q_\phi(z|x)$, **et** produit une prédiction de classe supervisée $z_o$, mais que, **pour les exemples négatifs**, il ne produit qu'une prédiction de classe $z_o$.

### Définition de la fonction de perte de la DB-VAE

Cela signifie que nous devrons être un peu plus malins sur la fonction de perte pour la DB-VAE. La forme de la perte dépendra de la question de savoir si l'on envisage une image de visage ou une image sans visage. 

Pour les **images de visages**, notre fonction de perte aura deux composantes :


1.   **perte EVA ($L_{EVA}$)** : comprend la perte latente et la perte de reconstruction.
2.   **Perte de classification ($L_y(y,\hat{y})$)** : perte d'entropie croisée standard pour un problème de classification binaire. 

En revanche, pour les images de **non-visages**, notre fonction de perte est uniquement la perte de classification. 

Nous pouvons écrire une seule expression pour la perte en définissant une variable indicatrice $\mathcal{I}_f$qui reflète quelles données d'entraînement sont des images de visages ($\mathcal{I}_f(y) = 1$ ) et quelles sont des images de **non-visages ($\mathcal{I}_f(y) = 0$). En utilisant ces données, nous obtenons :

$$L_{total} = L_y(y,\hat{y}) + \mathcal{I}_f(y)\Big[L_{VAE}\Big]$$

Ecrivons une fonction pour définir la fonction de perte de DB-VAE :



In [0]:
### Fonction de perte pour le DB-VAE ###

"""Fonction de perte pour le DB-VAE.
# Arguments
    x: vraie entrée x
    x_pred: x reconstruit
    y: vrai label (face ou non face)
    y_logit: labels prédits
    mu: moyenne de la distribution latente (Q(z|X))
    logsigma: log of standard deviation of latent distribution (Q(z|X))
# Retour
    total_loss: perte totale du DB-VAE
    classification_loss = perte de classification du DB-VAE
"""
def debiasing_loss_function(x, x_pred, y, y_logit, mu, logsigma):

  # A FAIRE : appeler la fonction concernée pour obtenir la perte de VAE
  vae_loss = vae_loss_function('''A FAIRE''') # A FAIRE

  # A FAIRE : définir la perte de classification en utilisant sigmoid_cross_entropy
  # https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits
  classification_loss = # A FAIRE

  # Utilisez les étiquettes de données d'apprentissage pour créer la variable face_indicator :
  #   indicateur qui reflète quelles données d'apprentissage sont des images de visages
  face_indicator = tf.cast(tf.equal(y, 1), tf.float32)

  # A FAIRE : définir la perte totale de la DB-VAE ! Utilisez tf.reduce_mean pour faire la moyenne
  # sur tous les échantillons
  total_loss = # A FAIRE

  return total_loss, classification_loss

### Architecture DB-VAE

Nous sommes maintenant prêts à définir l'architecture DB-VAE. Pour construire la DB-VAE, nous utiliserons le classificateur CNN standard ci-dessus comme codeur, puis nous définirons un réseau de décodeurs. Nous créerons et initialiserons les deux modèles, puis nous construirons le VAE de bout en bout. Nous utiliserons un espace latent avec 100 variables latentes.

Le réseau de décodeurs prendra comme entrée les variables latentes échantillonnées, les fera passer à travers une série de couches de déconvolution et produira une reconstruction de l'image d'entrée originale.

In [0]:
### Définir la partie décodeur du DB-VAE ###

n_filters = 12 # nombre de base des filtres de convolution, le même que dans le CNN standard
latent_dim = 100 # nombre de variables latentes

def make_face_decoder_network():
  # Définir fonctionnellement les différents types de couches que nous utiliserons
  Conv2DTranspose = functools.partial(tf.keras.layers.Conv2DTranspose, padding='same', activation='relu')
  BatchNormalization = tf.keras.layers.BatchNormalization
  Flatten = tf.keras.layers.Flatten
  Dense = functools.partial(tf.keras.layers.Dense, activation='relu')
  Reshape = tf.keras.layers.Reshape

  # Construire le réseau de décodeurs en utilisant l'API séquentielle
  decoder = tf.keras.Sequential([
    # Transformer en génération pré-convolutionnelle
    Dense(units=4*4*6*n_filters),  # 4x4 feature maps (with 6N occurances)
    Reshape(target_shape=(4, 4, 6*n_filters)),

    # Agrandissement des convolutions (inverse du codeur)
    Conv2DTranspose(filters=4*n_filters, kernel_size=3,  strides=2),
    Conv2DTranspose(filters=2*n_filters, kernel_size=3,  strides=2),
    Conv2DTranspose(filters=1*n_filters, kernel_size=5,  strides=2),
    Conv2DTranspose(filters=3, kernel_size=5,  strides=2),
  ])

  return decoder

Nous allons maintenant assembler ce décodeur avec le classificateur CNN standard comme notre codeur pour définir le DB-VAE. Notez qu'à ce stade, il n'y a rien de spécial dans la façon dont nous assemblons le modèle qui en fasse un modèle de "débiaisement" -- cela viendra lorsque nous définirons l'opération d'apprentissage. Ici, nous allons définir l'architecture centrale du VAE en sous-classant la classe "Model", en définissant les opérations d'encodage, de reparamétrage et de décodage, et en appelant le réseau de bout en bout..

In [0]:
### Définir et créer le DB-VAE ###

class DB_VAE(tf.keras.Model):
  def __init__(self, latent_dim):
    super(DB_VAE, self).__init__()
    self.latent_dim = latent_dim

    # Définissez le nombre de sorties pour le codeur. Rappelons que nous 
    # avons des variables latentes `latent_dim`, ainsi qu'une sortie  
    # supervisée pour la classification.
    num_encoder_dims = 2*self.latent_dim + 1

    self.encoder = make_standard_classifier(num_encoder_dims)
    self.decoder = make_face_decoder_network()

  # fonction d'alimentation des images dans le codeur, de codage
  #   de l'espace latent et de probabilité de classification de sortie 
  def encode(self, x):
    # encoder output
    encoder_output = self.encoder(x)

    # prédiction de classification
    y_logit = tf.expand_dims(encoder_output[:, 0], -1)
    # paramètres de distribution des variables latentes
    z_mean = encoder_output[:, 1:self.latent_dim+1] 
    z_logsigma = encoder_output[:, self.latent_dim+1:]

    return y_logit, z_mean, z_logsigma

  # Reparamétrage de la VAE : étant donné une moyenne et un logsigma, échantillon de variables latentes
  def reparameterize(self, z_mean, z_logsigma):
    # A FAIRE : appeler la fonction d'échantillonnage définie ci-dessus
    z = # A FAIRE
    return z

  # Décodage de l'espace latent et reconstruction de la sortie
  def decode(self, z):
    # A FAIRE : utiliser le décodeur pour produire la reconstruction
    reconstruction = # A FAIRE
    return reconstruction

  # La fonction call sera utilisée pour faire passer les entrées x par le noyau du VAE
  def call(self, x): 
    # Encoder l'entrée d'une prédiction et d'un espace latent
    y_logit, z_mean, z_logsigma = self.encode(x)

    # A FAIRE : reparamétrisation
    z = # A FAIRE

    # A FAIRE : reconstruction
    recon = # A FAIRE
    return y_logit, z_mean, z_logsigma, recon

  # Prédire le logit visage ou non visage pour une entrée donnée x
  def predict(self, x):
    y_logit, z_mean, z_logsigma = self.encode(x)
    return y_logit

dbvae = DB_VAE(latent_dim)

Comme indiqué, l'architecture du codeur est identique à celle de la CNN précédent dans ce laboratoire. Notez les sorties de notre modèle DB_VAE construit dans la fonction `call` : `y_logit, z_mean, z_logsigma, z`. Réfléchissez bien à la raison pour laquelle chacune de ces fonctions est produite et à leur importance pour le problème en question.



### Rééchantillonnage adaptatif pour un débiaisement automatisé avec DB-VAE

Alors, comment pouvons-nous réellement utiliser la DB-VAE pour former un classificateur de détection faciale débiaisé ?

Rappelez-vous l'architecture DB-VAE : lorsque les images d'entrée sont alimentées par le réseau, l'encodeur apprend une estimation $\mathcal{Q}(z|X)$ de l'espace latent. Nous voulons augmenter la fréquence relative des données rares en augmentant l'échantillonnage des régions sous-représentées de l'espace latent. Nous pouvons approximer $\mathcal{Q}(z|X)$ en utilisant les distributions de fréquence de chacune des variables latentes apprises, puis définir la distribution de probabilité de sélection d'un point de données donné $x$ sur la base de cette approximation. Ces distributions de probabilités seront utilisées pendant l'apprentissage pour ré-échantillonner les données.

Vous allez écrire une fonction pour exécuter cette mise à jour des probabilités d'échantillonnage, puis appeler cette fonction dans la boucle de formation DB-VAE pour débiaiser réellement le modèle. 

Tout d'abord, nous avons défini une courte fonction d'aide `get_latent_mu` qui renvoie les moyennes des variables latentes renvoyées par l'encodeur après l'entrée d'un lot d'images sur le réseau :

In [0]:
# Fonction permettant de renvoyer les moyennes pour un lot d'images d'entrée
def get_latent_mu(images, dbvae, batch_size=1024):
  N = images.shape[0]
  mu = np.zeros((N, latent_dim))
  for start_ind in range(0, N, batch_size):
    end_ind = min(start_ind+batch_size, N+1)
    batch = (images[start_ind:end_ind]).astype(np.float32)/255.
    _, batch_mu, _ = dbvae.encode(batch)
    mu[start_ind:end_ind] = batch_mu
  return mu

Maintenant, définissons l'algorithme de rééchantillonnage réel `get_training_sample_probabilities`. Notez bien l'argument `smoothing_fac`. Ce paramètre ajuste le degré de débiaisement : pour `smoothing_fac=0`, l'ensemble de formation ré-échantillonné aura tendance à s'appliquer uniformément sur l'espace latent, c'est-à-dire le débiaisement le plus extrême.

In [0]:
### Algorithme de rééchantillonnage pour le DB-VAE ###

'''Fonction qui recalcule les probabilités d'échantillonnage des images d'un lot en fonction 
      de leur répartition dans les données d'apprentissage'''
def get_training_sample_probabilities(images, dbvae, bins=10, smoothing_fac=0.001): 
    print("Recomputing the sampling probabilities")
    
    # A FAIRE : exécuter le lot d'entrée et obtenir les moyennes des variables latentes
    mu = get_latent_mu('''A FAIRE''') # A FAIRE

    # les probabilités d'échantillonnage des images
    training_sample_p = np.zeros(mu.shape[0])
    
    # considérer la distribution pour chaque variable latente 
    for i in range(latent_dim):
      
        latent_distribution = mu[:,i]
        # générer un histogramme de la distribution latente
        hist_density, bin_edges =  np.histogram(latent_distribution, density=True, bins=bins)

        # trouver dans quel conteneur latent se trouve chaque échantillon de données 
        bin_edges[0] = -float('inf')
        bin_edges[-1] = float('inf')
        
        # A FAIRE : call the digitize function to find which bins in the latent distribution 
        #    appeler la fonction de numérisation pour trouver dans quels conteneurs de la distribution latente se trouve chaque échantillon de données
        # https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.digitize.html
        bin_idx = np.digitize('''A FAIRE''', '''A FAIRE''') # A FAIRE

        # lisser la fonction de densité
        hist_smoothed_density = hist_density + smoothing_fac
        hist_smoothed_density = hist_smoothed_density / np.sum(hist_smoothed_density)

        # inverser la fonction de densité 
        p = 1.0/(hist_smoothed_density[bin_idx-1])
        
        # A FAIRE : normaliser toutes les probabilities
        p = # A FAIRE
        
        # A FAIRE : mettre à jour les probabilités d'échantillonnage en examinant si le p nouvellement calculé
        #     est supérieur aux probabilités d'échantillonnage existantes.
        training_sample_p = # TODO
        
    # normalisation finale
    training_sample_p /= np.sum(training_sample_p)

    return training_sample_p

Maintenant que nous avons défini la mise à jour du rééchantillonnage, nous pouvons former notre modèle DB-VAE sur les données d'apprentissage CelebA/ImageNet, et exécuter l'opération ci-dessus pour re-pondérer l'importance de points de données particuliers pendant que nous formons le modèle. Rappelez-vous encore une fois que nous ne voulons débusquer que les caractéristiques pertinentes pour les *faces*, et non l'ensemble des exemples négatifs. Complétez le bloc de code ci-dessous pour exécuter la boucle d'apprentissage !

In [0]:
### Entrainer le DB-VAE ###

# Hyperparamètres
batch_size = 32
learning_rate = 5e-4
latent_dim = 100

# DB-VAE a besoin d'un peu plus d'époques pour s'entraîner car il est plus complexe que le 
# classificateur standard, nous en utilisons donc 6 au lieu de 2
num_epochs = 6  

# instancier un nouveau modèle DB-VAE et un optimiseur
dbvae = DB_VAE(100)
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Pour définir l'opération d'apprentissage, nous utiliserons tf.function qui est un outil puissant qui 
#   nous permet de transformer une fonction Python en un graphe de calcul TensorFlow.
@tf.function
def debiasing_train_step(x, y):

  with tf.GradientTape() as tape:
    # Introduire l'entrée x dans dbvae. Notez que cela utilise la fonction call du DB_VAE !
    y_logit, z_mean, z_logsigma, x_recon = dbvae(x)

    '''A FAIRE : appeler la fonction de perte du DB_VAE pour calculer la perte'''
    loss, class_loss = debiasing_loss_function('''A FAIRE arguments''') # A FAIRE
  
  '''A FAIRE : utiliser la méthode GradientTape.gradient pour calculer les gradients.
     Indice : il s'agit des variables d'apprentissage du dbvae.'''
  grads = tape.gradient('''A FAIRE''', '''A FAIRE''') # A FAIRE

  # appliquer des gradients aux variables
  optimizer.apply_gradients(zip(grads, dbvae.trainable_variables))
  return loss

# obtenir des visages d'apprentissage à partir du chargeur de données
all_faces = loader.get_all_train_faces()

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

# La boucle d'apprentissage - la boucle extérieure itère sur le nombre d'époques
for i in range(num_epochs):

  IPython.display.clear_output(wait=True)
  print("Starting epoch {}/{}".format(i+1, num_epochs))

  # Recalculer les probabilités d'échantillonnage des données
  '''A FAIRE : recalculer les probabilités d'échantillonnage pour le débiaisement'''
  p_faces = get_training_sample_probabilities('''A FAIRE''', '''A FAIRE''') # A FAIRE
  
  # obtenir un lot de données d'apprentissage et calculer l'étape d'apprentissage
  for j in tqdm(range(loader.get_train_size() // batch_size)):
    # charger un lot de données
    (x, y) = loader.get_batch(batch_size, p_pos=p_faces)
    # optimisation de la perte
    loss = debiasing_train_step(x, y)
    
    # traçer la progression tous les 200 pas
    if j % 500 == 0: 
      mdl.util.plot_sample(x, y, dbvae)

Merveilleux ! Nous devrions maintenant disposer d'un modèle de classification du visage appris et (espérons-le !) débiaisé, prêt à être évalué !

## 2.6 Évaluation de la DB-VAE sur l'ensemble des données de test

Enfin, nous allons tester notre modèle DB-VAE sur l'ensemble des données de test, en examinant spécifiquement sa précision sur chacune des données démographiques "Homme foncé", "Femme foncée", "Homme clair" et "Femme claire". Nous comparerons les performances de ce modèle débiaisé à celles du modèle standard CNN (potentiellement biaisé) utilisé plus tôt dans le laboratoire.

In [0]:
dbvae_logits = [dbvae.predict(np.array(x, dtype=np.float32)) for x in test_faces]
dbvae_probs = tf.squeeze(tf.sigmoid(dbvae_logits))

xx = np.arange(len(keys))
plt.bar(xx, standard_classifier_probs.numpy().mean(1), width=0.2, label="Standard CNN")
plt.bar(xx+0.2, dbvae_probs.numpy().mean(1), width=0.2, label="DB-VAE")
plt.xticks(xx, keys); 
plt.title("Network predictions on test dataset")
plt.ylabel("Probability"); plt.legend(bbox_to_anchor=(1.04,1), loc="upper left");


## 2.7 Conclusion 

Nous vous encourageons à réfléchir et peut-être même à répondre à certaines questions soulevées par l'approche et les résultats décrits ici :

* Comment la précision de la DB-VAE dans les quatre catégories démographiques se compare-t-elle à celle de la CNN standard ? Ce résultat vous surprend-il d'une quelconque manière ?
* Comment améliorer encore les performances du classificateur DB-VAE ? Nous n'avons volontairement pas optimisé les hyperparamètres pour vous laisser le soin de le faire ! Si vous voulez aller plus loin, essayez d'optimiser votre modèle pour obtenir les meilleures performances. **[Envoyez-nous](mailto:introtodeeplearning-staff@mit.edu) une copie de votre notebook avec le diagramme à barres 2.6 exécuté, et nous donnerons des prix aux meilleurs ! 
* Dans quelles applications (liées à la détection faciale ou non !) un tel débiaisement serait-il souhaitable ? Y a-t-il des applications pour lesquelles vous ne souhaitez peut-être pas débiaiser votre modèle ? 
* Pensez-vous qu'il devrait être nécessaire pour les entreprises de démontrer que leurs modèles, en particulier dans le contexte de tâches comme la détection faciale, ne sont pas biaisés ? Si oui, avez-vous des idées sur la façon dont cela pourrait être normalisé et mis en œuvre ?
* Avez-vous des idées sur d'autres moyens de traiter les questions de biais, notamment en ce qui concerne les données d'apprentissage ?

Nous espérons que ce laboratoire a permis de faire la lumière sur quelques concepts, des tâches basées sur la vision, aux VAE, aux biais algorithmiques. Nous aimons à penser que c'est le cas, mais nous sommes biaisés ;). 

![Faces](https://media1.tenor.com/images/44e1f590924eca94fe86067a4cf44c72/tenor.gif?itemid=3394328)